Set your drive path here

In [1]:
drive_path = '/media/stephen/Brain20191/allen-brain-observatory/visual-coding-2p/'

In [2]:
import os
import pandas as pd
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.stimulus_info as stim_info
from utils.data_wrappers import *
manifest_file = os.path.join(drive_path,'manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_file)


Print the various values for brain area, layer, celltype

In [3]:
brain_areas = boc.get_all_targeted_structures()
brain_areas

['VISal', 'VISam', 'VISl', 'VISp', 'VISpm', 'VISrl']

In [4]:
depths = boc.get_all_imaging_depths()
depths

[175,
 185,
 195,
 200,
 205,
 225,
 250,
 265,
 275,
 276,
 285,
 300,
 320,
 325,
 335,
 350,
 365,
 375,
 390,
 400,
 550,
 570,
 625]

In [5]:
cell_types = boc.get_all_cre_lines()
cell_types

['Cux2-CreERT2',
 'Emx1-IRES-Cre',
 'Fezf2-CreER',
 'Nr5a1-Cre',
 'Ntsr1-Cre_GN220',
 'Pvalb-IRES-Cre',
 'Rbp4-Cre_KL100',
 'Rorb-IRES2-Cre',
 'Scnn1a-Tg3-Cre',
 'Slc17a7-IRES2-Cre',
 'Sst-IRES-Cre',
 'Tlx3-Cre_PL56',
 'Vip-IRES-Cre']

In [6]:
boc.get_all_stimuli()
# TF: 1,2,4,8,15

['drifting_gratings',
 'locally_sparse_noise',
 'locally_sparse_noise_4deg',
 'locally_sparse_noise_8deg',
 'natural_movie_one',
 'natural_movie_three',
 'natural_movie_two',
 'natural_scenes',
 'spontaneous',
 'static_gratings']

Experiment counts by brain area, layer, cell type

In [7]:
exps = boc.get_ophys_experiments(stimuli=[stim_info.DRIFTING_GRATINGS])
exps_df = pd.DataFrame.from_dict(exps)
exps_df.groupby(['targeted_structure', 'imaging_depth', 'cre_line'])['id'].count()

targeted_structure  imaging_depth  cre_line         
VISal               175            Cux2-CreERT2         6
                                   Emx1-IRES-Cre        2
                                   Slc17a7-IRES2-Cre    1
                    265            Emx1-IRES-Cre        1
                    275            Cux2-CreERT2         6
                                   Emx1-IRES-Cre        1
                                   Rorb-IRES2-Cre       6
                                   Slc17a7-IRES2-Cre    1
                                   Sst-IRES-Cre         1
                    276            Cux2-CreERT2         1
                    300            Nr5a1-Cre            1
                    350            Nr5a1-Cre            5
                    365            Emx1-IRES-Cre        1
                    375            Emx1-IRES-Cre        2
                                   Rbp4-Cre_KL100       6
VISam               175            Cux2-CreERT2         5
                   

Show cell counts before and after filtering.
See https://pastebin.com/NGzvE0Ky for result

In [8]:
exps = boc.get_ophys_experiments(stimuli=[stim_info.DRIFTING_GRATINGS])
exps = pd.DataFrame.from_dict(exps)
cells = boc.get_cell_specimens()
cells = pd.DataFrame.from_records(cells)
dg_response_filter = get_dg_response_filter_from_saskia()

for brain_area in brain_areas:
    for depth in depths:
        for cell_type in boc.get_all_cre_lines():
            if len(exps[
                (exps['targeted_structure'] == brain_area) &
                (exps['imaging_depth'] == depth) &
                (exps['cre_line'] == cell_type)
                ]) == 0:
                continue
            
            in_category_cells = get_cells(boc, cells, brain_area, depth, cell_type)
            
            if len(in_category_cells) == 0:
                continue
            cell_count_before_responsive_filter = len(in_category_cells)
            responsive_cells = get_filtered_cells(in_category_cells, dg_response_filter)
            cell_count_after_responsive_filter = len(responsive_cells)
            print (brain_area,
                   depth,
                   cell_type,
                   cell_count_before_responsive_filter,
                   cell_count_after_responsive_filter )

VISal 175 Cux2-CreERT2 1226 371
VISal 175 Emx1-IRES-Cre 512 180
VISal 175 Slc17a7-IRES2-Cre 64 19
VISal 265 Emx1-IRES-Cre 346 162
VISal 275 Cux2-CreERT2 1665 727
VISal 275 Emx1-IRES-Cre 492 251
VISal 275 Rorb-IRES2-Cre 1242 389
VISal 275 Slc17a7-IRES2-Cre 310 104
VISal 275 Sst-IRES-Cre 24 21
VISal 276 Cux2-CreERT2 212 88
VISal 300 Nr5a1-Cre 78 24
VISal 350 Nr5a1-Cre 142 49
VISal 365 Emx1-IRES-Cre 162 69
VISal 375 Emx1-IRES-Cre 275 118
VISal 375 Rbp4-Cre_KL100 441 155
VISam 175 Cux2-CreERT2 619 155
VISam 175 Emx1-IRES-Cre 167 49
VISam 175 Slc17a7-IRES2-Cre 235 51
VISam 275 Cux2-CreERT2 997 339
VISam 275 Emx1-IRES-Cre 195 22
VISam 275 Rorb-IRES2-Cre 735 252
VISam 320 Nr5a1-Cre 5 2
VISam 325 Nr5a1-Cre 75 24
VISam 350 Nr5a1-Cre 91 13
VISam 375 Emx1-IRES-Cre 95 30
VISam 375 Rbp4-Cre_KL100 355 136
VISl 175 Cux2-CreERT2 1208 399
VISl 175 Emx1-IRES-Cre 1122 453
VISl 175 Slc17a7-IRES2-Cre 1115 342
VISl 175 Vip-IRES-Cre 161 13
VISl 250 Sst-IRES-Cre 56 40
VISl 250 Vip-IRES-Cre 27 0
VISl 275 Cux2-